In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle

from my_classes import DataGeneratorSkillBorders

2024-06-11 11:34:11.631096: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 11:34:11.673434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 11:34:12.402992: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
video_border_labels_path = '../data/df_video_border_labels.pkl'
video_folder = '../videos/'

video_names = [
    '20240201_atelier_001.mp4',
    '20240201_atelier_002.mp4',
    '20240201_atelier_003.mp4',
    '20240201_atelier_004.mp4',
    '20240201_atelier_005.mp4',
    '20240209_atelier_006.mp4',
    '20240209_atelier_007.mp4',
    '20240209_atelier_008.mp4',
]

train_videos = [ video_folder + trainvid for trainvid in video_names]

In [5]:
def pickle_load_or_create(path, cols):
    if os.path.exists(path):
        with open(path, 'rb') as file:
            return pickle.load(file)
    else:
        return pd.DataFrame(columns=cols)

In [6]:
df_labels = pickle_load_or_create(video_border_labels_path, [])
df_labels

,path,frame,border
0,20240209_atelier_006.mp4,745,0
1,20240209_atelier_007.mp4,103,0
2,20240201_atelier_002.mp4,91,0
3,20240201_atelier_004.mp4,806,2
4,20240201_atelier_003.mp4,622,2
...,...,...,...
733,20240201_atelier_002.mp4,549,0
734,20240201_atelier_003.mp4,491,0
735,20240201_atelier_003.mp4,290,2
736,20240201_atelier_001.mp4,346,0


In [7]:
df_labels.loc[df_labels.border == 5, 'border'] = 3
df_labels.loc[df_labels.border == 9, 'border'] = 4

In [8]:
def get_random_frame(videos, grey=True, scale=1):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
          0 : ground
          1 : heels of ground
          2 : air
          3 : Fault
          4 : no skipper or not jumping
    """
    path = videos[np.random.randint(0, len(videos)-1)]
    
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # frame = cv2.cvtColor(frame, 7)
    cap.release()
    # cv2.destroyAllWindows()

    return path, frame_nr, frame

In [9]:
input_shape = get_random_frame(train_videos, grey=True, scale=0.4)[2].shape
input_shape

(768, 432, 3)

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,320 (220.00 KB)

 Trainable params: 56,320 (220.00 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
unique_labels = df_labels['border'].unique()
unique_labels

array([0, 2, 1, 4, 3])

In [13]:
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5))

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       147,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,877 (796.39 KB)

 Trainable params: 203,877 (796.39 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from my_classes import DataGeneratorSkillBorders
# Parameters
params = {'dim': (64,64),
          'batch_size': 16,
          'n_classes': len(unique_labels),
          'n_channels': 3,
          'shuffle': True,
          'train': True}

training_generator = DataGeneratorSkillBorders(df_labels, video_folder=video_folder, **params)
test_generator = DataGeneratorSkillBorders(df_labels, video_folder=video_folder, train=False)

                         path  frame  border
267  20240201_atelier_003.mp4   1105       2
447  20240201_atelier_005.mp4   1449       0
615  20240201_atelier_003.mp4    312       0
185  20240201_atelier_005.mp4   1945       0
526  20240209_atelier_007.mp4   1109       0
..                        ...    ...     ...
636  20240209_atelier_006.mp4    831       4
19   20240201_atelier_003.mp4   1335       4
730  20240201_atelier_001.mp4    633       4
500  20240201_atelier_001.mp4    303       2
306  20240201_atelier_004.mp4   1748       1

[590 rows x 3 columns]
                         path  frame  border
628  20240209_atelier_007.mp4   1138       0
337  20240201_atelier_005.mp4   2318       0
564  20240209_atelier_007.mp4    194       0
149  20240201_atelier_005.mp4   1640       3
346  20240201_atelier_005.mp4    109       4
..                        ...    ...     ...
30   20240201_atelier_005.mp4   1405       0
461  20240209_atelier_007.mp4    135       1
615  20240201_atelier_003.mp4  

In [16]:
%%time
X, y = training_generator.__getitem__(3)

CPU times: user 12.3 s, sys: 246 ms, total: 12.6 s
Wall time: 3.12 s


In [17]:
X

array([[[[0.27058824, 0.28235294, 0.28235294],
         [0.27843137, 0.29019608, 0.29019608],
         [0.29803922, 0.30980392, 0.30980392],
         ...,
         [0.25882353, 0.27058824, 0.27058824],
         [0.25882353, 0.27058824, 0.27058824],
         [0.26666667, 0.27843137, 0.27843137]],

        [[0.29411765, 0.30588235, 0.30588235],
         [0.29411765, 0.30588235, 0.30588235],
         [0.28235294, 0.29411765, 0.29411765],
         ...,
         [0.2745098 , 0.28627451, 0.28627451],
         [0.27843137, 0.29019608, 0.29019608],
         [0.28627451, 0.29803922, 0.29803922]],

        [[0.32941176, 0.34117647, 0.34117647],
         [0.32156863, 0.33333333, 0.33333333],
         [0.29019608, 0.30196078, 0.30196078],
         ...,
         [0.28627451, 0.29803922, 0.29803922],
         [0.28235294, 0.29411765, 0.29411765],
         [0.27058824, 0.28235294, 0.28235294]],

        ...,

        [[0.74117647, 0.70980392, 0.6627451 ],
         [0.71372549, 0.68627451, 0.62745098]

In [18]:
y

array([2, 0, 3, 1, 0, 2, 4, 4, 0, 0, 0, 2, 2, 4, 2, 1])

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=10, 
                    validation_data=test_generator)

Epoch 1/10


2024-06-11 11:34:36.909134: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3 of 8
2024-06-11 11:34:54.850821: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 7 of 8


 1/36 ━━━━━━━━━━━━━━━━━━━━ 20:58 36s/step - accuracy: 0.3750 - loss: 1.5454

2024-06-11 11:34:59.320669: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


35/36 ━━━━━━━━━━━━━━━━━━━━ 3s 4s/step - accuracy: 0.4653 - loss: 1.4140 

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 9216, but received input with shape (None, 50176)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 128, 128, 3), dtype=float32)
  • training=False
  • mask=None

In [ ]:
pd.DataFrame(history.history)

In [ ]:
with open('../models/last_cnn_model_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)